In [1]:
import numpy as np
from numbers import Number
from pyscf import gto, scf, ao2mo, fci
from copy import deepcopy, copy
from math import factorial
import time

#define vacuum
#take in operator string
#format: string of indices, bitstring of creation vs annihilation
#return normal-ordered operator string plus contracted terms, according to Wick's theorem



In [3]:
class Index:
    '''
    Class for an orbital index
    name (str): name given to index
    occupiedInVaccum (bool): whether the index refers to orbitals that are occupied in the vacuum (i.e. hole orbitals) or not (particle orbitals)
    '''
    def __init__(self, name, occupiedInVacuum):
        self.name = name
        self.occupiedInVacuum = occupiedInVacuum
        self.tuple = (self.name, self.occupiedInVacuum)

    def __hash__(self):
        return hash(self.tuple)

    def __eq__(self, other):
        return self.tuple == other

    def __str__(self):
        return self.name

In [4]:
class basicOperator:
    '''
    Class for a basic fermionic creation or annihilation operator, with arbitrary index
    index (Index): the index for the orbitals on which this operator acts
    creation_annihilation (bool): True for creation, False for annihilation
    spin (bool): True for alpha, False for beta
    quasi_cre_ann (bool): creation or annihilation with respect to Fermi vacuum
    '''
    def __init__(self, index_, creation_annihilation_, spin_):
        self.index = index_
        self.spin = spin_
        self.creation_annihilation = creation_annihilation_
        self.quasi_cre_ann = not (self.creation_annihilation == self.index.occupiedInVacuum)

    def __copy__(self):
        return basicOperator(self.index, self.creation_annihilation, self.spin)

    def __str__(self):
        string = "a"
        if bool(self.creation_annihilation):
            string = string + "^"
        else:
            string = string + "_"
        if bool(self.spin):
            string = string + "{" + self.index.__str__() + "\\alpha}"
        else:
            string = string + "{" + self.index.__str__() + "\\beta}"
        return string

    def __eq__(self, other):
        if isinstance(other, basicOperator):
            return self.index == other.index and self.spin == other.spin and self.creation_annihilation == other.creation_annihilation
        return False

    def conjugate(self):
        return basicOperator(self.index, not self.creation_annihilation, self.spin)

In [5]:
class operatorProduct:
    def __init__(self, operatorList_=[], prefactor_=1., contractionsList_=[]):
        self.operatorList = operatorList_
        self.prefactor = prefactor_
        self.contractionsList = contractionsList_

    def isProportional(self, other):
        if isinstance(other, operatorProduct):
            return self.operatorList == other.operatorList and set(self.contractionsList) == set(other.contractionsList)
        else:
            return NotImplemented

    def __copy__(self):
        return operatorProduct(copy(self.operatorList), self.prefactor, copy(self.contractionsList))

    def __str__(self):
        string = str(self.prefactor)
        if(len(self.operatorList) + len(self.contractionsList) > 0):
            string = string + " * "
        for o in self.operatorList:
            string = string + o.__str__()
        for contraction in self.contractionsList:
            string = string + "\delta^{" + contraction[0].__str__() + "}_{" + contraction[1].__str__() + "}"
        return string

    def __mul__(self, other):
        if isinstance(other, operatorProduct):
            return operatorProduct(self.operatorList + other.operatorList, self.prefactor * other.prefactor, self.contractionsList + other.contractionsList)
        elif isinstance(other, operatorSum):
            newSummandList = []
            for s in other.summandList:
                newSummandList.append(self * s)
            return operatorSum(newSummandList)
        elif isinstance(other, Number):
            return operatorProduct(self.operatorList, self.prefactor * other, self.contractionsList)
        else:
            return NotImplemented

    def __rmul__(self, other):
        if isinstance(other, Number):
            return operatorProduct(self.operatorList, other * self.prefactor, self.contractionsList)
        else:
            return NotImplemented

    def __add__(self, other):
        if isinstance(other, operatorProduct):
            if self.isProportional(other):
                return operatorProduct(self.operatorList, self.prefactor + other.prefactor, self.contractionsList)
            else:
                return operatorSum([self, other])
        elif other == 0:
            return self
        else:
            return NotImplemented

    def __radd__(self, other):
        if other == 0:
            return self
        else:
            return NotImplemented

    def __eq__(self, other):
        if isinstance(other, operatorProduct):
            return self.operatorList == other.operatorList and self.prefactor == other.prefactor and self.contractionsList == other.contractionsList
        else:
            return NotImplemented
            
    def checkNilpotency(self):
        nonZero = True
        i = 0
        while i < len(self.operatorList):
            j = i + 1
            while j < len(self.operatorList):
                if self.operatorList[j] == self.operatorList[i]:
                    nonZero = False
                elif self.operatorList[j] == self.operatorList[i].conjugate():
                    break
                j = j + 1
            i = i + 1
        return nonZero

    def conjugate(self):
        return operatorProduct([o.conjugate() for o in self.operatorList], np.conjugate(self.prefactor), self.contractionsList)

In [6]:
class operatorSum:
    def __init__(self, summandList_=[]):
        self.summandList = self.collectSummandList(summandList_)

    def collectSummandList(self, summandList):
        oldSummandList = copy(summandList)
        newSummandList = []
        while len(oldSummandList) > 0:
            newSummand = oldSummandList[0]
            i = 1
            while i < len(oldSummandList):
                if newSummand.isProportional(oldSummandList[i]):
                    newSummand += oldSummandList[i]
                    oldSummandList.pop(i)
                else:
                    i += 1
            newSummandList.append(newSummand)
            oldSummandList.pop(0)
        return newSummandList

    def __copy__(self):
        return operatorSum(copy(self.summandList))

    def __str__(self):
        if len(self.summandList) == 0:
            return str(0)
        string = self.summandList[0].__str__()
        s = 1
        while s < len(self.summandList):
            string = string + "\n + " + self.summandList[s].__str__()
            s = s + 1
        return string

    def __add__(self, other):
        if isinstance(other, operatorSum):
            return operatorSum(self.summandList + other.summandList)
        elif isinstance(other, operatorProduct):
            if other.prefactor == 0:
                return self
            newSummandList = copy(self.summandList)
            alreadyInSum = False
            for summand in newSummandList:
                if summand.isProportional(other):
                    alreadyInSum = True
                    summand.prefactor += other.prefactor
            if not alreadyInSum:
                newSummandList.append(other)
            return operatorSum(newSummandList)
        elif other == 0:
            return self
        else:
            return NotImplemented
            
    def __radd__(self, other):
        if isinstance(other, operatorProduct):
            if other.prefactor == 0:
                return self
            newSummandList = copy(self.summandList)
            alreadyInSum = False
            for summand in newSummandList:
                if summand.isProportional(other):
                    alreadyInSum = True
                    summand.prefactor += other.prefactor
            if not alreadyInSum:
                newSummandList.append(other)
            return operatorSum(newSummandList)
        elif other == 0:
            return self
        else:
            return NotImplemented

    def __mul__(self, other):
        if isinstance(other, operatorProduct):
            newSummandList = []
            for s in self.summandList:
                newSummandList.append(s * other)
            return operatorSum(newSummandList)
        elif isinstance(other, operatorSum):
            newSummandList = []
            for o in other.summandList:
                partialSum = self * o
                newSummandList = newSummandList + partialSum.summandList
            return operatorSum(newSummandList)
        elif isinstance(other, Number):
            return operatorSum([self.summandList[s] * other for s in range(len(self.summandList))])
        else:
            return NotImplemented

    def __rmul__(self, other):
        if isinstance(other, operatorProduct):
            newSummandList = []
            for s in self.summandList:
                newSummandList.append(other * s)
            return operatorSum(newSummandList)
        elif isinstance(other, Number):
            return operatorSum([other * self.summandList[s] for s in range(len(self.summandList))])
        else:
            return NotImplemented

    def conjugate(self):
        return operatorSum([s.conjugate() for s in self.summandList])

In [7]:
def normalOrder(operator):
    '''
    Input: an operatorProduct or operatorSum and a list corresponding to which orbitals are occupied in the Fermi vacuum
    Output: normal ordered form of input, with respect to vacuum
    '''
    if isinstance(operator, operatorSum):
        return operatorSum([normalOrder(product) for product in operator.summandList])
    quasiCreationList, quasiAnnihilationList = [], []
    quasiCreationCount = 0
    sign = 1
    for o in range(len(operator.operatorList)):
        op = operator.operatorList[o]
        if bool(op.quasi_cre_ann):
            quasiCreationList.append(op)
            if (o - quasiCreationCount) % 2 == 1:
                sign = -sign
            quasiCreationCount += 1
        else:
            quasiAnnihilationList.append(op)
    return operatorProduct(quasiCreationList + quasiAnnihilationList, sign * operator.prefactor, operator.contractionsList)

In [9]:
def anticommute(operatorProduct_, first):
    '''
    Apply fermionic anticommutation relation to two adjacent second-quantized operators in an operatorProduct
    '''
    operatorList_ = operatorProduct_.operatorList
#    firstIndex = operatorList_[first].orbital
#    secondIndex = operatorList_[first + 1].orbital
    commutedTerm = operatorProduct(operatorList_[:first] + operatorList_[first + 1] + operatorList_[first] + operatorList_[first + 1:], operatorProduct_.prefactor * (-1))
#    if operatorList_[first].quasi_cre_ann == operatorList_[second].quasi_cre_ann:
    return commutedTerm
#    elif firstIndex == secondIndex:
#        contractedTerm = operatorProduct(operatorList_[:first] + operatorList_[first+2:], operatorProduct_.prefactor)
#        return operatorSum([contractedTerm, commutedTerm])
#    else:
#        return commutedTerm

In [10]:
def anticommuteInPlace(operatorProduct_, first):
    '''
    Apply fermionic anticommutation relation in place to two adjacent second-quantized operators in an operatorProduct
    '''
    operatorList_ = operatorProduct_.operatorList
    if operatorList_ == []:
        return
    firstOperatorCopy = copy(operatorList_[first])
    operatorList_[first], operatorList_[first + 1] = operatorList_[first + 1], firstOperatorCopy
    operatorProduct_.prefactor = -operatorProduct_.prefactor
    return

In [11]:
def contract(operatorProduct_, first, second):
    '''
    Make a contraction between the indices of operators at the positions given by first and second in operatorProduct_
    '''
    operatorList_ = operatorProduct_.operatorList
    existingContractions = operatorProduct_.contractionsList
    if operatorList_ == []:
        return operatorProduct_
    firstIndex = operatorList_[first].index
    secondIndex = operatorList_[second].index
#    if firstIndex == secondIndex:
    if firstIndex.occupiedInVacuum != secondIndex.occupiedInVacuum:
        return operatorProduct([],0)
    if operatorList_[first].spin != operatorList_[second].spin:
        return operatorProduct([],0)
    if operatorList_[first].quasi_cre_ann:
        return operatorProduct([],0)
    if not operatorList_[second].quasi_cre_ann:
        return operatorProduct([],0)
    else:
        contractionTuple = tuple()
        if operatorList_[first].creation_annihilation:
            contractionTuple = (firstIndex, secondIndex)
        else:
            contractionTuple = (secondIndex, firstIndex)
        return operatorProduct(operatorList_[:first] + operatorList_[first+1:second] + operatorList_[second + 1:], operatorProduct_.prefactor * ((-1) ** (1 + second - first)), existingContractions + [contractionTuple])

In [12]:
def reorderForContraction(operatorProduct_, first, second):
    '''
    Reorder operators using anticommutation relation so that two operators to be contracted are in positions 0 and 1
    '''
    operatorProductCopy = copy(operatorProduct_)
    i = first
    j = second
    while i > 0:
        anticommuteInPlace(operatorProductCopy, i-1)
        i = i - 1
    while j > 1:
        anticommuteInPlace(operatorProductCopy, j-1)
        j = j - 1
    return operatorProductCopy


In [13]:
def reorderForMultipleContraction(operatorProduct_, pairsList):
    '''
    Reorder operators using anticommutation relation so that each pair of operators to be contracted are adjacent
    '''
    operatorProductCopy = copy(operatorProduct_)
    finalPosition = 0
    for positionPair in pairsList:
        i = positionPair[0]
        j = positionPair[1]
        while i > finalPosition:
            anticommuteInPlace(operatorProductCopy, i-1)
            i = i - 1
        while j > finalPosition + 1:
            anticommuteInPlace(operatorProductCopy, j-1)
            j = j - 1
        finalPosition = finalPosition + 2
    return operatorProductCopy

In [14]:
def updateFlattenedPositionList(flattenedPositions):
    for i in range(len(flattenedPositions)):
        if flattenedPositions[i] > flattenedPositions[1]:
            flattenedPositions[i] -= 2
        elif flattenedPositions[i] > flattenedPositions[0]:
            flattenedPositions[i] -= 1
    return flattenedPositions[2:]

In [15]:
def multipleContraction(operatorProduct_, positionPairsList):
    '''
    Contractions between multiple pairs of operators in a product, at the positions given by entries 2i and 2i+1 in positionPairsList
    '''
    product = copy(operatorProduct_)
    while len(positionPairsList) > 0:
#    for positionPair in pairsList:
        reorderedProduct = reorderForContraction(product, positionPairsList[0], positionPairsList[1])
        product = contract(reorderedProduct, 0, 1)
        positionPairsList = updateFlattenedPositionList(positionPairsList) #NOTE: reduces length of positionPairsList by 2
    return product

In [16]:
def nFoldContractionFromStart(operatorProduct_, n):
    product = copy(operatorProduct_)
    if (2 * n <= len(product.operatorList)):
        for i in range(n):
            product = contract(product, 0, 1)
    return product

In [8]:
oP = operatorProduct([basicOperator(Index('p', False),1,1), basicOperator(Index('p', False),0,1)])
nOP = normalOrder(oP)
for i in range(len(nOP.operatorList)):
    print(nOP.operatorList[i])

print(oP)
print(nOP)

print((nOP + oP) * (nOP + oP))

a = basicOperator(Index('p', False),1,1)
b = basicOperator(Index('p', False),1,1)

print(a==b)

print((nOP * nOP).checkNilpotency())

a^{p\alpha}
a_{p\alpha}
1.0 * a^{p\alpha}a_{p\alpha}
1.0 * a^{p\alpha}a_{p\alpha}
4.0 * a^{p\alpha}a_{p\alpha}a^{p\alpha}a_{p\alpha}
True
True


In [17]:
oP2 = operatorProduct([basicOperator(Index('a', False),0,1), basicOperator(Index('b', False),0,1), basicOperator(Index('a', False),1,1), basicOperator(Index('b', False),1,1)])
nOP2 = normalOrder(oP2)
print(oP2)
print(nOP2)

print("----")

print(contract(oP2, 0,2))
print(contract(oP2, 1,3))
print(contract(contract(oP2, 0,2),0,1))
print(nFoldContractionFromStart(oP2, 1))
print(len(oP2.operatorList))

print("----")

print(reorderForContraction(oP2,0,2))
print(nFoldContractionFromStart(reorderForContraction(oP2,0,2), 1))

print("----")

print(multipleContraction(oP2, []))

1.0 * a_{a\alpha}a_{b\alpha}a^{a\alpha}a^{b\alpha}
1.0 * a^{a\alpha}a^{b\alpha}a_{a\alpha}a_{b\alpha}
----
-1.0 * a_{b\alpha}a^{b\alpha}\delta^{a}_{a}
-1.0 * a_{a\alpha}a^{a\alpha}\delta^{b}_{b}
-1.0 * \delta^{a}_{a}\delta^{b}_{b}
0
4
----
-1.0 * a_{a\alpha}a^{a\alpha}a_{b\alpha}a^{b\alpha}
-1.0 * a_{b\alpha}a^{b\alpha}\delta^{a}_{a}
----
1.0 * a_{a\alpha}a_{b\alpha}a^{a\alpha}a^{b\alpha}


In [2]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff, compact=False)

cisolver = fci.FCI(mol, mf.mo_coeff)

Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1 for i in range(Nocc)] + [0 for i in range (Norbs - Nocc)]

In [18]:
positionPairs = [(0,2),(1,3)]
flattenedPositions = [pos for pair in positionPairs for pos in pair]
print(flattenedPositions)
flattenedPositionsAfterContraction = copy(flattenedPositions)
for i in range(len(flattenedPositionsAfterContraction)):
    if flattenedPositionsAfterContraction[i] > flattenedPositionsAfterContraction[1]:
        flattenedPositionsAfterContraction[i] -= 2
    elif flattenedPositionsAfterContraction[i] > flattenedPositionsAfterContraction[0]:
        flattenedPositionsAfterContraction[i] -= 1
print(flattenedPositionsAfterContraction[2:])
print(flattenedPositions)

[0, 2, 1, 3]
[0, 1]
[0, 2, 1, 3]


In [19]:
def genPairOrderedLists(list_):
    '''
    Given an ordered list with an even number of elements, returns list of all permutations of that list where each adjacent pair is ordered, and where the first elements of pairs are ordered
    '''
    pairOrderedLists = []
    if len(list_) == 2:
        pairOrderedLists.append(list_)
    else:
        for i in range(1,len(list_)):
            swappedList = [list_[0]] + [list_[i]] + list_[1:i]
            if i < len(list_):
                swappedList = swappedList + list_[i+1:]
            subLists = genPairOrderedLists(swappedList[2:])
            for l in range(len(subLists)):
                pairOrderedLists.append(swappedList[:2] + subLists[l])
    return pairOrderedLists

In [20]:
objectList = ["a", "b", "c", "d", "e", "f"]
print(genPairOrderedLists(objectList))

[['a', 'b', 'c', 'd', 'e', 'f'], ['a', 'b', 'c', 'e', 'd', 'f'], ['a', 'b', 'c', 'f', 'd', 'e'], ['a', 'c', 'b', 'd', 'e', 'f'], ['a', 'c', 'b', 'e', 'd', 'f'], ['a', 'c', 'b', 'f', 'd', 'e'], ['a', 'd', 'b', 'c', 'e', 'f'], ['a', 'd', 'b', 'e', 'c', 'f'], ['a', 'd', 'b', 'f', 'c', 'e'], ['a', 'e', 'b', 'c', 'd', 'f'], ['a', 'e', 'b', 'd', 'c', 'f'], ['a', 'e', 'b', 'f', 'c', 'd'], ['a', 'f', 'b', 'c', 'd', 'e'], ['a', 'f', 'b', 'd', 'c', 'e'], ['a', 'f', 'b', 'e', 'c', 'd']]


In [21]:
from itertools import combinations

In [22]:
def getPositionsForMultipleContraction(operatorProduct_, n):
    nOperators = len(operatorProduct_.operatorList)
    operatorPositions = range(nOperators)
    nToChoose = 2 * n
    if nToChoose <= nOperators:
        return combinations(operatorPositions, nToChoose)


In [23]:
print(oP2)
chosenPositions = getPositionsForMultipleContraction(oP2, 2)
print([list(c) for c in chosenPositions])
print([genPairOrderedLists(list(d)) for d in chosenPositions])

1.0 * a_{a\alpha}a_{b\alpha}a^{a\alpha}a^{b\alpha}
[[0, 1, 2, 3]]
[]


In [24]:
def sumNFoldContractions(operatorProduct_, n):
    chosenPositions = getPositionsForMultipleContraction(operatorProduct_, n)
    if n == 0:
        return operatorProduct_
    operatorSum_ = operatorSum([])
    for c in chosenPositions:
        pairOrderedList = genPairOrderedLists(list(c))
        for l in pairOrderedList:
            operatorSum_ = operatorSum_ + multipleContraction(operatorProduct_, l)
    return operatorSum_

In [25]:
print(oP2)
print(multipleContraction(oP2, [0,2,1,3]))
print(sumNFoldContractions(oP2, 1))

1.0 * a_{a\alpha}a_{b\alpha}a^{a\alpha}a^{b\alpha}
-1.0 * \delta^{a}_{a}\delta^{b}_{b}
-1.0 * a_{b\alpha}a^{b\alpha}\delta^{a}_{a}
 + 1.0 * a_{b\alpha}a^{a\alpha}\delta^{b}_{a}
 + 1.0 * a_{a\alpha}a^{b\alpha}\delta^{a}_{b}
 + -1.0 * a_{a\alpha}a^{a\alpha}\delta^{b}_{b}


In [26]:
def wickExpand(operator):
    if isinstance(operator, operatorSum):
        wickExpansion = operatorSum([])
        for product in operator.summandList:
            wickExpansion = wickExpansion + wickExpand(product)
        return wickExpansion
    wickExpansion = operatorSum([normalOrder(operator)])
    highestOrder = len(operator.operatorList) // 2
    for n in range(highestOrder):
        wickExpansion = wickExpansion + sumNFoldContractions(operator, n + 1)
    return normalOrder(wickExpansion)

In [27]:
def vacuumExpectationValue(operator):
    wickExpansion = wickExpand(operator)
    vEV = operatorSum()
    for summand in wickExpansion.summandList:
        if summand.operatorList == []:
            topologyAlreadyFound = False
            for topology in vEV.summandList:
                if set(summand.contractionsList) == set(topology.contractionsList):
                    topology.prefactor += summand.prefactor
                    topologyAlreadyFound = True
            if not topologyAlreadyFound:
                vEV += summand
    return vEV

In [28]:
print(oP2)
print(wickExpand(oP2))
print(vacuumExpectationValue(oP2))

1.0 * a_{a\alpha}a_{b\alpha}a^{a\alpha}a^{b\alpha}
1.0 * a^{a\alpha}a^{b\alpha}a_{a\alpha}a_{b\alpha}
 + 1.0 * a^{b\alpha}a_{b\alpha}\delta^{a}_{a}
 + -1.0 * a^{a\alpha}a_{b\alpha}\delta^{b}_{a}
 + -1.0 * a^{b\alpha}a_{a\alpha}\delta^{a}_{b}
 + 1.0 * a^{a\alpha}a_{a\alpha}\delta^{b}_{b}
 + -1.0 * \delta^{a}_{a}\delta^{b}_{b}
 + 1.0 * \delta^{b}_{a}\delta^{a}_{b}
-1.0 * \delta^{a}_{a}\delta^{b}_{b}
 + 1.0 * \delta^{b}_{a}\delta^{a}_{b}


In [29]:
#Particle number-conserving excitation operators, normal ordered with respect to true vacuum
class excitation(operatorProduct):
    def __init__(self, creationIndicesList_, annihilationIndicesList_, spinList_):
        super(excitation, self).__init__()
        self.operatorList = []
        for i in range(len(creationIndicesList_)):
            self.operatorList.append(basicOperator(creationIndicesList_[i], True, spinList_[i]))
        for i in range(len(annihilationIndicesList_)):
            self.operatorList.append(basicOperator(annihilationIndicesList_[i], False, spinList_[i+len(creationIndicesList_)]))

In [30]:
class spinFreeExcitation(operatorSum):
    def __init__(self, creationList_, annihilationList_):
        self.summandList = []
        spinLists = np.reshape(np.zeros(len(creationList_)), (1, -1))
        for i in range(len(creationList_)):
            newspinLists = copy(spinLists)
            for s in range(len(spinLists)):
                newspinLists[s,i] = 1
            spinLists = np.concatenate((spinLists, newspinLists))
        for l in range(len(spinLists)):
            spinList = spinLists[l]
            self.summandList.append(excitation(creationList_, annihilationList_, [*spinList, *spinList[::-1]]))

In [31]:
aI = Index('a', True)
bI = Index('b', True)
cI = Index('c', True)
dI = Index('d', True)
rI = Index('r', False)
sI = Index('s', False)
tI = Index('t', False)
uI = Index('u', False)

In [32]:
testOperator = spinFreeExcitation([rI], [aI])

In [33]:
print(testOperator)

1.0 * a^{r\beta}a_{a\beta}
 + 1.0 * a^{r\alpha}a_{a\alpha}


In [34]:
print(normalOrder(testOperator))

1.0 * a^{r\beta}a_{a\beta}
 + 1.0 * a^{r\alpha}a_{a\alpha}


In [35]:
print(wickExpand(testOperator))

1.0 * a^{r\beta}a_{a\beta}
 + 1.0 * a^{r\alpha}a_{a\alpha}


In [36]:
testOperator1 = spinFreeExcitation([sI], [bI])

In [37]:
print(testOperator.conjugate() * testOperator1)

1.0 * a_{r\beta}a^{a\beta}a^{s\beta}a_{b\beta}
 + 1.0 * a_{r\alpha}a^{a\alpha}a^{s\beta}a_{b\beta}
 + 1.0 * a_{r\beta}a^{a\beta}a^{s\alpha}a_{b\alpha}
 + 1.0 * a_{r\alpha}a^{a\alpha}a^{s\alpha}a_{b\alpha}


In [38]:
print(wickExpand(testOperator.conjugate() * testOperator1))

1.0 * a^{s\beta}a_{b\beta}a_{r\beta}a^{a\beta}
 + 1.0 * a_{b\beta}a^{a\beta}\delta^{s}_{r}
 + 1.0 * a^{s\beta}a_{r\beta}\delta^{a}_{b}
 + -2.0 * \delta^{s}_{r}\delta^{a}_{b}
 + 1.0 * a^{s\beta}a_{b\beta}a_{r\alpha}a^{a\alpha}
 + 1.0 * a^{s\alpha}a_{b\alpha}a_{r\beta}a^{a\beta}
 + 1.0 * a^{s\alpha}a_{b\alpha}a_{r\alpha}a^{a\alpha}
 + 1.0 * a_{b\alpha}a^{a\alpha}\delta^{s}_{r}
 + 1.0 * a^{s\alpha}a_{r\alpha}\delta^{a}_{b}


In [39]:
def canContract(o1, o2):
    if o1.quasi_cre_ann:
        return 0
    elif o2.quasi_cre_ann and (o1.spin == o2.spin):
        return int(o1.index.occupiedInVacuum == o2.index.occupiedInVacuum)
    else:
        return 0

In [40]:
def recursiveFullContraction(operatorProduct_, speedup=False):
#    if isinstance(operatorProduct_, Number):
#        return operatorProduct_
    operatorList_ = operatorProduct_.operatorList
    if speedup:
        if not sum(o.quasi_cre_ann for o in operatorList_) == sum(not o.quasi_cre_ann for o in operatorList_):
            return 0
    existingContractions = operatorProduct_.contractionsList
    if len(operatorList_) == 0:
        return operatorProduct_
    elif len(operatorList_) == 2:
        if canContract(operatorList_[0], operatorList_[1]):
            contractionTuple = tuple()
            if operatorList_[0].creation_annihilation:
                contractionTuple = (operatorList_[0].index, operatorList_[1].index)
            else:
                contractionTuple = (operatorList_[1].index, operatorList_[0].index)
            return operatorProduct([], operatorProduct_.prefactor, existingContractions + [contractionTuple])
        else:
            return 0
    elif len(operatorList_) % 2 == 0:
        result = 0
        for i in range(1, len(operatorList_) - 1):
            if canContract(operatorList_[0], operatorList_[i]):
                contractionTuple = tuple()
                if operatorList_[0].creation_annihilation:
                    contractionTuple = (operatorList_[0].index, operatorList_[i].index)
                else:
                    contractionTuple = (operatorList_[i].index, operatorList_[0].index)
                result += pow(-1, i-1) * recursiveFullContraction(operatorProduct(operatorList_[1:i] + operatorList_[i+1:], operatorProduct_.prefactor, existingContractions + [contractionTuple]))
        if canContract(operatorList_[0], operatorList_[-1]):
            contractionTuple = tuple()
            if operatorList_[0].creation_annihilation:
                contractionTuple = (operatorList_[0].index, operatorList_[-1].index)
            else:
                contractionTuple = (operatorList_[-1].index, operatorList_[0].index)
            result += recursiveFullContraction(operatorProduct(operatorList_[1:-1], operatorProduct_.prefactor, existingContractions + [contractionTuple]))
        return result
    else:
        return 0

In [41]:
print((testOperator.conjugate() * testOperator1).summandList[0])

1.0 * a_{r\beta}a^{a\beta}a^{s\beta}a_{b\beta}


In [42]:
print(recursiveFullContraction((testOperator.conjugate() * testOperator1).summandList[0]))

-1.0 * \delta^{s}_{r}\delta^{a}_{b}


In [43]:
def recursiveVacuumExpectationValue(operator, speedup=False, printing=False):
    if isinstance(operator, operatorProduct):
        return recursiveFullContraction(operator, speedup)
    elif isinstance(operator, operatorSum):
        result = operatorSum([])
        for product in operator.summandList:
            term = recursiveFullContraction(product, speedup)
            if printing and term != 0.:
                print(product)
            result += term
        return result
    elif isinstance(operator, Number):
        return operatorSum([operatorProduct([], operator)])
    else:
        return operatorSum([])

In [44]:
print(vacuumExpectationValue(testOperator.conjugate() * testOperator1).summandList[0].contractionsList)

[(<__main__.Index object at 0x121070f70>, <__main__.Index object at 0x121070e80>), (<__main__.Index object at 0x121070040>, <__main__.Index object at 0x1210709a0>)]


In [125]:
class Tensor:
    '''
    Class for amplitude tensors of spin-free excitation operators
    '''
#    def __init__(self, name, lowerIndexTypesList, upperIndexTypesList, array=None):
    def __init__(self, name, lowerIndexTypesList, upperIndexTypesList):
        self.name = name
        self.lowerIndexTypes = lowerIndexTypesList
        self.upperIndexTypes = upperIndexTypesList
        self.excitationRank = len(self.lowerIndexTypes)
#        self.array = np.zeros(self.getShape())
#        if array is not None:
#            self.setArray(array)

    def getShape(self, vacuum):
        Norbs = len(vacuum)
        Nocc = sum(vacuum)
        shapeList = []
        for iType in self.lowerIndexTypes:
            if iType == 'g':
                shapeList.append(Norbs)
            elif iType == 'p':
                shapeList.append(Norbs - Nocc)
            elif iType == 'h':
                shapeList.append(Nocc)
            else:
                print('Orbital index type Error')
        for iType in self.upperIndexTypes:
            if iType == 'g':
                shapeList.append(Norbs)
            elif iType == 'p':
                shapeList.append(Norbs - Nocc)
            elif iType == 'h':
                shapeList.append(Nocc)
            else:
                print('Orbital index type Error')
#        return tuple(shapeList)
        self.array = np.zeros(tuple(shapeList))

    def setArray(self, array):
        if array.shape == self.array.shape:
            self.array = array
        else:
            print("Array is of wrong shape")

    def getOperator(self):
        return TensorProduct([self]).getOperator()
#        operator = operatorSum([])
#        it = np.nditer(self.array, flags=["multi_index"])
#        
#        for i in it:
#            indices = it.multi_index
#            creationIndices = indices[:self.excitationRank]
#            annihilationIndices = indices[self.excitationRank:]
#            operator += i * spinFreeExcitation(creationIndices, annihilationIndices)
#        return operator

    def getDiagrams(self):
        diagrams = []
        lowerGeneralIndexCount = sum(i == 'g' for i in self.lowerIndexTypes)
        lowerSplits = list(itertools.combinations_with_replacement(['h', 'p'], lowerGeneralIndexCount))
        upperGeneralIndexCount = sum(i == 'g' for i in self.upperIndexTypes)
        upperSplits = list(itertools.combinations_with_replacement(['h', 'p'], upperGeneralIndexCount))
        for lowerSplit in lowerSplits:
            lowerSlices = [slice(None)] * self.excitationRank
            lowerSplitIndexTypes = list(lowerSplit)
            lGI = 0
            newLowerIndexTypes = copy.copy(self.lowerIndexTypes)
            for lI in range(len(newLowerIndexTypes)):
                if newLowerIndexTypes[lI] == 'g':
                    newLI = lowerSplitIndexTypes[lGI]
                    if newLI == 'h':
                        lowerSlices[lI] = slice(None,Nocc)
                    elif newLI == 'p':
                        lowerSlices[lI] = slice(Nocc, None)
                    newLowerIndexTypes[lI] = newLI
                    lGI += 1
            for upperSplit in upperSplits:
                upperSlices = [slice(None)] * self.excitationRank
                upperSplitIndexTypes = list(upperSplit)
                uGI = 0
                newUpperIndexTypes = copy.copy(self.upperIndexTypes)
                for uI in range(len(newUpperIndexTypes)):
                    if newUpperIndexTypes[uI] == 'g':
                        newUI = upperSplitIndexTypes[uGI]
                        if newUI == 'h':
                            upperSlices[uI] = slice(None,Nocc)
                        elif newUI == 'p':
                            upperSlices[uI] = slice(Nocc, None)
                        newUpperIndexTypes[uI] = newUI
                        uGI += 1
                slices = lowerSlices + upperSlices
                print(lowerSplitIndexTypes)
                print(upperSplitIndexTypes)
                print(newLowerIndexTypes)
                print(newUpperIndexTypes)
                print(slices)
                diagram = Tensor(self.name, newLowerIndexTypes, newUpperIndexTypes, self.array[slices])
                diagrams.append(diagram)
        return diagrams

    def __add__(self, other):
        if isinstance(other, Tensor):
            return TensorSum([TensorProduct([self]), TensorProduct([other])])
        elif other == 0:
            return self
        else:
            return NotImplemented

    def __radd__(self, other):
        if other == 0:
            return self
        else:
            return NotImplemented

    def __mul__(self, other):
        if isinstance(other, Tensor):
            return TensorProduct([self, other])
        elif isinstance(other, Number):
            return TensorProduct([self], other)
        else:
            return NotImplemented

    def __rmul__(self, other):
        if isinstance(other, Number):
            return TensorProduct([self], other)
        else:
            return NotImplemented

    def __str__(self):
        string = self.name + "_{"
        for p in self.lowerIndexTypes:
            string += p.__str__()
        string += "}^{"
        for q in self.upperIndexTypes:
            string += q.__str__()
        string += "}"
        return string

In [46]:
class Vertex:
    '''
    Class for amplitude tensors of spin-free excitation operators
    '''
    def __init__(self, tensor, lowerIndicesList, upperIndicesList):
        self.name = tensor.name
        self.tensor = tensor
        self.lowerIndices = lowerIndicesList
        self.upperIndices = upperIndicesList
        self.excitationRank = len(self.lowerIndices)

    def applyContraction(self, contraction):
        for lowerIndex in self.lowerIndices:
            if lowerIndex == contraction[0]:
                lowerIndex = contraction[1]

    def getOperator(self):
        return spinFreeExcitation(self.lowerIndices, self.upperIndices)

    def __copy__(self):
        return vertex(self.tensor, self.lowerIndices, self.upperIndices)

    def __str__(self):
        string = self.name + "_{"
        for p in self.lowerIndices:
            string += p.__str__()
        string += "}^{"
        for q in self.upperIndices:
            string += q.__str__()
        string += "}"
        return string

In [47]:
class TensorProduct:
    def __init__(self, tensorList, prefactor=1., vertexList=None):
        self.tensorList = tensorList
        self.lowerIndices = {'g':[], 'p':[], 'h':[], 'a':[]}
        self.upperIndices = {'g':[], 'p':[], 'h':[], 'a':[]}
        self.prefactor = prefactor
        if vertexList is None:
            self.vertexList = self.getVertexList(tensorList)

    def applyContraction(self, contraction):
        for vertex in self.vertexList:
            vertex.applyContraction(contraction)
        for orbitalType in self.lowerIndices:
            if contraction[0] in self.lowerIndices[orbitalType]:
                self.lowerIndices[orbitalType].remove(contraction[0])

    def addNewIndex(self, orbitalType, lowerBool):
        count = len(self.lowerIndices[orbitalType]) + len(self.upperIndices[orbitalType])
        newIndexName = orbitalType + "_{" + str(count) + "}"
        occupiedInVacuum = None
        if orbitalType == "h" or orbitalType == "a":
            occupiedInVacuum = True
        elif orbitalType == "p":
            occupiedInVacuum = False
        newIndex = Index(newIndexName, occupiedInVacuum)
        if lowerBool:
            self.lowerIndices[orbitalType].append(newIndex)
        else:
            self.upperIndices[orbitalType].append(newIndex)
        return newIndex

    def getVertexList(self, tensorList_):
        vertexList = []
        for t in tensorList_:
            lowerIndexList = []
            for i in t.lowerIndexTypes:
                lowerIndexList.append(self.addNewIndex(i, True))
            upperIndexList = []
            for i in t.upperIndexTypes:
                upperIndexList.append(self.addNewIndex(i, False))
            vertexList.append(Vertex(t, lowerIndexList, upperIndexList))
        return vertexList

    def getOperator(self):
        operator = 1
        for vertex in self.vertexList:
            operator = operator * vertex.getOperator()
        return operator

    def getVacuumExpectationValue(self):
        return vacuumExpectationValue(self.getOperator())

    def __copy__(self):
        return TensorProduct(copy(self.tensorList), self.prefactor)

    def __add__(self, other):
        if isinstance(other, TensorProduct):
            return TensorSum([self, other])
        elif isinstance(other, Tensor):
            return TensorSum([self, TensorProduct([other])])
        else:
            return NotImplemented

    def __radd__(self, other):
        if isinstance(other, Tensor):
            return TensorSum([TensorProduct([other]), self])
        elif other == 0:
            return self
        else:
            return NotImplemented

    def __mul__(self, other):
        if isinstance(other, Tensor):
            return TensorProduct(self.tensorList + [other], self.prefactor)
        elif isinstance(other, TensorProduct):
            return TensorProduct(self.tensorList + other.tensorList, self.prefactor * other.prefactor)
        elif isinstance(other, Number):
            return TensorProduct(self.tensorList, self.prefactor * other)
        else:
            return NotImplemented

    def __rmul__(self, other):
        if isinstance(other, Tensor):
            return TensorProduct([other] + self.tensorList, self.prefactor)
        elif isinstance(other, Number):
            return TensorProduct(self.tensorList, other * self.prefactor)
        else:
            return NotImplemented

    def __str__(self):
        string = str(self.prefactor)
        if(len(self.vertexList) > 0):
            string = string + " * "
        for v in self.vertexList:
            string += v.__str__()
        return string

In [48]:
class TensorSum:
    def __init__(self, summandList):
        self.summandList = summandList

    def getOperator(self):
        operator = 0
        for summand in self.summandList:
            operator = operator + summand.getOperator()
        return operator

    def __copy__(self):
        return TensorSum(copy(self.summandList))

    def __add__(self, other):
        if isinstance(other, TensorSum):
            return TensorSum(self.summandList + other.summandList)
        elif isinstance(other, TensorProduct):
            return TensorSum(self.summandList + [other])
        elif isinstance(other, Tensor):
            return TensorSum(self.summandList + [TensorProduct([other])])
        else:
            return NotImplemented

    def __radd__(self, other):
        if isinstance(other, TensorProduct):
            return TensorSum([other] + self.summandList)
        elif isinstance(other, Tensor):
            return TensorSum(self.summandList + [TensorProduct([other])])
        elif other == 0:
            return self
        else:
            return NotImplemented

    def __mul__(self, other):
        if isinstance(other, Tensor) or isinstance(other, TensorProduct) or isinstance(other, TensorSum) or isinstance(other, Number):
            return TensorSum([summand * other for summand in self.summandList])

    def __rmul__(self, other):
        if isinstance(other, Tensor) or isinstance(other, TensorProduct) or isinstance(other, TensorSum) or isinstance(other, Number):
            return TensorSum([other * summand for summand in self.summandList])

    def __str__(self):
        if len(self.summandList) == 0:
            return ""
        string = self.summandList[0].__str__()
        for summand in self.summandList[1:]:
            string += "\n + "
            string += summand.__str__()
        return string

In [236]:
fDiagrams = [Tensor('f', ['p'], ['p']), Tensor('f', ['p'], ['h']), Tensor('f', ['h'], ['p']), Tensor('f', ['h'], ['h'])]

In [237]:
vDiagrams = [Tensor('v', ['p', 'p'], ['p', 'p']), Tensor('v', ['h', 'h'], ['h', 'h']), Tensor('v', ['h', 'p'], ['p', 'h']), Tensor('v', ['p', 'h'], ['p', 'p']), Tensor('v', ['h', 'h'], ['h', 'p']), Tensor('v', ['p', 'p'], ['p', 'h']), Tensor('v', ['h', 'p'], ['h', 'h']), Tensor('v', ['p', 'p'], ['h', 'h']), Tensor('v', ['h', 'h'], ['p', 'p'])]

In [238]:
t1 = Tensor('t1', ['p'], ['h'])
t2 = Tensor('t2', ['p', 'p'], ['h', 'h'])

In [52]:
term = vDiagrams[8] * t2
#print(term, vacuumExpectationValue(term.getOperator()))

In [53]:
fullContractions = vacuumExpectationValue(term.getOperator())

In [54]:
recursiveFullContractions = recursiveVacuumExpectationValue(term.getOperator())

In [55]:
for fC in fullContractions.summandList:
    print(fC)

4.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{2}}_{p_{0}}\delta^{p_{3}}_{p_{1}}
-2.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{3}}_{p_{0}}\delta^{p_{2}}_{p_{1}}
-2.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{2}}_{p_{0}}\delta^{p_{3}}_{p_{1}}
4.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{3}}_{p_{0}}\delta^{p_{2}}_{p_{1}}


In [56]:
print(fullContractions)

4.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{2}}_{p_{0}}\delta^{p_{3}}_{p_{1}}
 + -2.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{3}}_{p_{0}}\delta^{p_{2}}_{p_{1}}
 + -2.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{2}}_{p_{0}}\delta^{p_{3}}_{p_{1}}
 + 4.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{3}}_{p_{0}}\delta^{p_{2}}_{p_{1}}


In [57]:
print(recursiveFullContractions)

4.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{2}}_{p_{0}}\delta^{p_{3}}_{p_{1}}
 + -2.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{3}}_{p_{0}}\delta^{p_{2}}_{p_{1}}
 + -2.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{2}}_{p_{0}}\delta^{p_{3}}_{p_{1}}
 + 4.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{3}}_{p_{0}}\delta^{p_{2}}_{p_{1}}


In [58]:
print(term)

1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}


In [59]:
contractedTerm = copy(term)

In [60]:
top = recursiveFullContractions.summandList[0]

In [61]:
print(top)

4.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{2}}_{p_{0}}\delta^{p_{3}}_{p_{1}}


In [62]:
#contractedTerm.prefactor *= top.prefactor
for v, vertex in enumerate(term.vertexList):
    print(vertex)
    for i, index in enumerate(vertex.lowerIndices):
        print(index)
        for c, contraction in enumerate(top.contractionsList):
            print(contraction[0], contraction[1])
            if term.vertexList[v].lowerIndices[i] == top.contractionsList[c][0]:
                contractedTerm.vertexList[v].lowerIndices[i] = top.contractionsList[c][1]

v_{h_{0}h_{1}}^{p_{0}p_{1}}
h_{0}
h_{0} h_{2}
h_{1} h_{3}
p_{2} p_{0}
p_{3} p_{1}
h_{1}
h_{0} h_{2}
h_{1} h_{3}
p_{2} p_{0}
p_{3} p_{1}
t2_{p_{2}p_{3}}^{h_{2}h_{3}}
p_{2}
h_{0} h_{2}
h_{1} h_{3}
p_{2} p_{0}
p_{3} p_{1}
p_{3}
h_{0} h_{2}
h_{1} h_{3}
p_{2} p_{0}
p_{3} p_{1}


In [63]:
print(contractedTerm)

1.0 * v_{h_{2}h_{3}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{3}}


In [64]:
contractedTerm = top.prefactor * contractedTerm

In [65]:
for c, contraction in enumerate(top.contractionsList):
    print(contraction[0], contraction[1])
    contractedTerm.applyContraction(contraction)

h_{0} h_{2}
h_{1} h_{3}
p_{2} p_{0}
p_{3} p_{1}


In [66]:
print(top.contractionsList[0][0])

h_{0}


In [67]:
print(term.vertexList[0].lowerIndices[0] == top.contractionsList[0][0])

True


In [68]:
contractedTerm.vertexList[0].lowerIndices[0] = top.contractionsList[0][1]

In [69]:
print(contractedTerm.vertexList[0].lowerIndices[0])

h_{2}


In [70]:
print(contractedTerm)

4.0 * v_{h_{2}h_{1}}^{p_{0}p_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}


In [71]:
contractionsList = fullContractions.summandList[0].contractionsList

In [72]:
fullContractions.summandList[0].prefactor

4.0

In [73]:
contractedTerm = copy(term) * 4

In [74]:
contractedTerm.vertexList[0].lowerIndices

[<__main__.Index at 0x121046a90>, <__main__.Index at 0x10e8ae370>]

In [75]:
print(term)

1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}


In [76]:
term.prefactor

1.0

In [77]:
term = term * 4

In [78]:
for vertex in contractedTerm.vertexList:
    for i in range(len(vertex.lowerIndices)):
        for contraction in contractionsList:
            print(contraction[0], contraction[1])
            if vertex.lowerIndices[i] == contraction[0]:
                vertex.lowerIndices[i] = contraction[1]

h_{0} h_{2}
h_{1} h_{3}
p_{2} p_{0}
p_{3} p_{1}
h_{0} h_{2}
h_{1} h_{3}
p_{2} p_{0}
p_{3} p_{1}
h_{0} h_{2}
h_{1} h_{3}
p_{2} p_{0}
p_{3} p_{1}
h_{0} h_{2}
h_{1} h_{3}
p_{2} p_{0}
p_{3} p_{1}


In [79]:
print(contractedTerm)

4.0 * v_{h_{2}h_{3}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{3}}


In [80]:
del contractedTerm

In [205]:
def evaluateWick(term):
    '''
    Wick's theorem applied to a term

    input: term (TensorProduct)
    output: sum of fully contracted terms (TensorSum)
    '''
    if isinstance(term, TensorSum):
        return sum([evaluateWick(summand) for summand in term.summandList])
    summandList = []
    fullContractions = recursiveVacuumExpectationValue(term.getOperator(), speedup=True)
    for topology in fullContractions.summandList:
        contractionsList = topology.contractionsList
        prefactor = topology.prefactor
        contractedTerm = copy(term)
        contractedTerm.prefactor *= topology.prefactor
        for c, contraction in enumerate(contractionsList):
            for v, vertex in reversed(list(enumerate(term.vertexList))):
                if contraction[0] in vertex.lowerIndices:
                    contractedTerm.vertexList[v].lowerIndices[vertex.lowerIndices.index(contraction[0])] = contraction[1]
                    break
                elif contraction[1] in vertex.upperIndices:
                    contractedTerm.vertexList[v].upperIndices[vertex.upperIndices.index(contraction[1])] = contraction[0]
                    break
#            for i, index in enumerate(vertex.lowerIndices):
#        for v, vertex in reversed(list(enumerate(term.vertexList))):
#            for i, index in reversed(list(enumerate(vertex.lowerIndices))):
#                for c, contraction in enumerate(topology.contractionsList):
#                    if index == contraction[0]:
#                        contractedTerm.vertexList[v].lowerIndices[i] = contraction[1]
        summandList.append(contractedTerm)
    return TensorSum(summandList)


In [195]:
CCDEnergyTerms = sum([0.5 * d * t2 for d in vDiagrams])

In [196]:
contractions = recursiveVacuumExpectationValue((vDiagrams[8] * t2).getOperator())

In [197]:
print(contractions.summandList[1])

-2.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{3}}_{p_{0}}\delta^{p_{2}}_{p_{1}}


In [204]:
for term in CCDEnergyTerms.summandList:
    print(term)

0.5 * v_{p_{0}p_{1}}^{p_{2}p_{3}}t2_{p_{4}p_{5}}^{h_{0}h_{1}}
0.5 * v_{h_{0}h_{1}}^{h_{2}h_{3}}t2_{p_{0}p_{1}}^{h_{4}h_{5}}
0.5 * v_{h_{0}p_{0}}^{p_{1}h_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}
0.5 * v_{p_{0}h_{0}}^{p_{1}p_{2}}t2_{p_{3}p_{4}}^{h_{1}h_{2}}
0.5 * v_{h_{0}h_{1}}^{h_{2}p_{0}}t2_{p_{1}p_{2}}^{h_{3}h_{4}}
0.5 * v_{p_{0}p_{1}}^{p_{2}h_{0}}t2_{p_{3}p_{4}}^{h_{1}h_{2}}
0.5 * v_{h_{0}p_{0}}^{h_{1}h_{2}}t2_{p_{1}p_{2}}^{h_{3}h_{4}}
0.5 * v_{p_{0}p_{1}}^{h_{0}h_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}
0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}


In [206]:
CCDEnergy = evaluateWick(CCDEnergyTerms)

In [200]:
print(recursiveVacuumExpectationValue(CCDEnergyTerms.getOperator()))

4.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{2}}_{p_{0}}\delta^{p_{3}}_{p_{1}}
 + -2.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{3}}_{p_{0}}\delta^{p_{2}}_{p_{1}}
 + -2.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{2}}_{p_{0}}\delta^{p_{3}}_{p_{1}}
 + 4.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{3}}_{p_{0}}\delta^{p_{2}}_{p_{1}}


In [207]:
print(CCDEnergy)

2.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{1}}
 + -1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{1}h_{0}}
 + 2.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{1}h_{0}}


In [239]:
def commutator(operator1, operator2):
    return operator1 * operator2 + (-1) * operator2 * operator1

In [89]:
print(commutator(commutator(vDiagrams[8], t2), t2))

1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}t2_{p_{4}p_{5}}^{h_{4}h_{5}}
 + -1 * t2_{p_{0}p_{1}}^{h_{0}h_{1}}v_{h_{2}h_{3}}^{p_{2}p_{3}}t2_{p_{4}p_{5}}^{h_{4}h_{5}}
 + -1.0 * t2_{p_{0}p_{1}}^{h_{0}h_{1}}v_{h_{2}h_{3}}^{p_{2}p_{3}}t2_{p_{4}p_{5}}^{h_{4}h_{5}}
 + 1 * t2_{p_{0}p_{1}}^{h_{0}h_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}v_{h_{4}h_{5}}^{p_{4}p_{5}}


In [240]:
def BCHSimilarityTransform(H, T, order):
    result = H
    for k in range(order):
        nestedCommutator = H
        for i in range(k + 1):
            nestedCommutator = commutator(nestedCommutator, T)
        result += (1 / factorial(k + 1)) * nestedCommutator
    return result

In [91]:
print(sum(vDiagrams))

1.0 * v_{p_{0}p_{1}}^{p_{2}p_{3}}
 + 1.0 * v_{h_{0}h_{1}}^{h_{2}h_{3}}
 + 1.0 * v_{h_{0}p_{0}}^{p_{1}h_{1}}
 + 1.0 * v_{p_{0}h_{0}}^{p_{1}p_{2}}
 + 1.0 * v_{h_{0}h_{1}}^{h_{2}p_{0}}
 + 1.0 * v_{p_{0}p_{1}}^{p_{2}h_{0}}
 + 1.0 * v_{h_{0}p_{0}}^{h_{1}h_{2}}
 + 1.0 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + 1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}


In [241]:
transformH = sum([BCHSimilarityTransform(d, (1./4.) * t2, 4) for d in fDiagrams] + [BCHSimilarityTransform((1./2.) * d, (1./4.) * t2, 4) for d in vDiagrams])

In [93]:
print(transformH)

0.25 * f_{p_{0}}^{p_{1}}t2_{p_{2}p_{3}}^{h_{0}h_{1}}
 + -0.25 * t2_{p_{0}p_{1}}^{h_{0}h_{1}}f_{p_{2}}^{p_{3}}
 + 1.0 * f_{p_{0}}^{p_{1}}
 + 0.03125 * f_{p_{0}}^{p_{1}}t2_{p_{2}p_{3}}^{h_{0}h_{1}}t2_{p_{4}p_{5}}^{h_{2}h_{3}}
 + -0.03125 * t2_{p_{0}p_{1}}^{h_{0}h_{1}}f_{p_{2}}^{p_{3}}t2_{p_{4}p_{5}}^{h_{2}h_{3}}
 + -0.03125 * t2_{p_{0}p_{1}}^{h_{0}h_{1}}f_{p_{2}}^{p_{3}}t2_{p_{4}p_{5}}^{h_{2}h_{3}}
 + 0.03125 * t2_{p_{0}p_{1}}^{h_{0}h_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}f_{p_{4}}^{p_{5}}
 + 0.0026041666666666665 * f_{p_{0}}^{p_{1}}t2_{p_{2}p_{3}}^{h_{0}h_{1}}t2_{p_{4}p_{5}}^{h_{2}h_{3}}t2_{p_{6}p_{7}}^{h_{4}h_{5}}
 + -0.0026041666666666665 * t2_{p_{0}p_{1}}^{h_{0}h_{1}}f_{p_{2}}^{p_{3}}t2_{p_{4}p_{5}}^{h_{2}h_{3}}t2_{p_{6}p_{7}}^{h_{4}h_{5}}
 + -0.0026041666666666665 * t2_{p_{0}p_{1}}^{h_{0}h_{1}}f_{p_{2}}^{p_{3}}t2_{p_{4}p_{5}}^{h_{2}h_{3}}t2_{p_{6}p_{7}}^{h_{4}h_{5}}
 + 0.0026041666666666665 * t2_{p_{0}p_{1}}^{h_{0}h_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}f_{p_{4}}^{p_{5}}t2_{p_{6}p_{7}}^{h_{4}h_

In [94]:
print(transformH.summandList[402])

8.138020833333333e-05 * t2_{p_{0}p_{1}}^{h_{0}h_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}t2_{p_{4}p_{5}}^{h_{4}h_{5}}t2_{p_{6}p_{7}}^{h_{6}h_{7}}v_{h_{8}h_{9}}^{p_{8}p_{9}}


In [95]:
print(t2 * t2)

1.0 * t2_{p_{0}p_{1}}^{h_{0}h_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}


In [208]:
time0 = time.time()
first = evaluateWick(vDiagrams[8] * t2)
time1 = time.time()
print(time1-time0)
print(first)

0.004615277997800149
4.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -2.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{1}}
 + -2.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{1}h_{0}}
 + 4.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{1}h_{0}}


In [209]:
time0 = time.time()
second = evaluateWick(vDiagrams[8] * t2 * t2)
time1 = time.time()
print(time1-time0)
print(second)

0.003674377003335394



In [210]:
type(vDiagrams[8])

__main__.Tensor

In [211]:
product = vDiagrams[8] * t2 * t2 * t2 * t2
op = product.getOperator()
time0 = time.time()
firstContractions = recursiveVacuumExpectationValue(op.summandList[0])
time1 = time.time()
print(time1-time0)
print(firstContractions)

0.016796209994936362
0


In [212]:
time0 = time.time()
CCDEnergyLastTerm = evaluateWick(transformH.summandList[402])
time1 = time.time()
print(time1-time0)
print(CCDEnergyLastTerm)

1.072306223999476



In [242]:
time0 = time.time()
CCDEnergy = evaluateWick(transformH)
time1 = time.time()
print(time1-time0)
print(CCDEnergy)

161.05956860099832
2.0 * f_{h_{1}}^{h_{1}}
 + -1.0 * v_{h_{2}h_{3}}^{h_{2}h_{3}}
 + 2.0 * v_{h_{3}h_{2}}^{h_{2}h_{3}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{1}}
 + -0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{1}h_{0}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{1}h_{0}}


In [102]:
print(recursiveVacuumExpectationValue((vDiagrams[8] * t2).getOperator()))

4.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{2}}_{p_{0}}\delta^{p_{3}}_{p_{1}}
 + -2.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{3}}_{p_{0}}\delta^{p_{2}}_{p_{1}}
 + -2.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{2}}_{p_{0}}\delta^{p_{3}}_{p_{1}}
 + 4.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{3}}_{p_{0}}\delta^{p_{2}}_{p_{1}}


In [103]:
print(evaluateWick(vDiagrams[8] * t2))

4.0 * v_{h_{2}h_{3}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{3}}
 + -2.0 * v_{h_{2}h_{3}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{2}h_{3}}
 + -2.0 * v_{h_{3}h_{2}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{3}}
 + 4.0 * v_{h_{3}h_{2}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{2}h_{3}}


In [214]:
E = evaluateWick(vDiagrams[8] * t2)

In [215]:
print(E.summandList[3].tensorList)

[<__main__.Tensor object at 0x11ff551c0>, <__main__.Tensor object at 0x12181efa0>]


In [216]:
t2

In [106]:
projectionSingles = Tensor('\Phi', ['h'], ['p'])
projectionDoubles = Tensor('\Phi', ['h', 'h'], ['p', 'p'])

In [107]:
def projectionManifold(excitationLevel):
    return Tensor('\Phi', ['h'] * excitationLevel, ['p'] * excitationLevel)

In [108]:
print(projectionManifold(2))

\Phi_{hh}^{pp}


In [109]:
time0 = time.time()
CCDDoublesResidual = evaluateWick(projectionManifold(2) *  transformH)
time1 = time.time()
print(time1-time0)
print(CCDDoublesResidual)

2308.2655238360003
-0.5 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{3}}
 + 1.0 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{2}h_{3}}
 + 1.0 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{2}h_{3}}
 + -0.5 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{2}h_{3}}
 + 1.0 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{3}}
 + -0.5 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{2}h_{3}}
 + -0.5 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{2}h_{3}}
 + 1.0 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{2}h_{3}}
 + 0.5 * \Phi_{h_{3}h_{4}}^{p_{0}p_{1}}f_{h_{5}}^{h_{3}}t2_{p_{0}p_{1}}^{h_{4}h_{5}}
 + -1.0 * \Phi_{h_{3}h_{4}}^{p_{0}p_{1}}f_{h_{5}}^{h_{3}}t2_{p_{1}p_{0}}^{h_{4}h_{5}}
 + -1.0 * \Phi_{h_{3}h_{5}}^{p_{0}p_{1}}f_{h_{4}}^{h_{3}}t2_{p_{0}p_{1}}^{h_{4}h_{5}}
 + 0.5 * \Phi_{h_{3}h_{5}}^{p_{0}p_{1}}f_{h

In [110]:
print(time1-time0)

2308.2655238360003


In [182]:
CCDDoublesResidual.summandList[0].tensorList

In [111]:
for s in recursiveVacuumExpectationValue((projectionManifold(2) * transformH.summandList[0]).getOperator(), speedup=False).summandList:
    print(s)

-2.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{2}}_{p_{0}}\delta^{p_{4}}_{p_{1}}\delta^{p_{5}}_{p_{3}}
4.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{2}}_{p_{0}}\delta^{p_{5}}_{p_{1}}\delta^{p_{4}}_{p_{3}}
4.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{4}}_{p_{0}}\delta^{p_{2}}_{p_{1}}\delta^{p_{5}}_{p_{3}}
-2.0 * \delta^{h_{0}}_{h_{2}}\delta^{h_{1}}_{h_{3}}\delta^{p_{5}}_{p_{0}}\delta^{p_{2}}_{p_{1}}\delta^{p_{4}}_{p_{3}}
4.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{2}}_{p_{0}}\delta^{p_{4}}_{p_{1}}\delta^{p_{5}}_{p_{3}}
-2.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{2}}_{p_{0}}\delta^{p_{5}}_{p_{1}}\delta^{p_{4}}_{p_{3}}
-2.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{4}}_{p_{0}}\delta^{p_{2}}_{p_{1}}\delta^{p_{5}}_{p_{3}}
4.0 * \delta^{h_{0}}_{h_{3}}\delta^{h_{1}}_{h_{2}}\delta^{p_{5}}_{p_{0}}\delta^{p_{2}}_{p_{1}}\delta^{p_{4}}_{p_{3}}


In [112]:
print(CCDDoublesResidual)

-0.5 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{3}}
 + 1.0 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{2}h_{3}}
 + 1.0 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{2}h_{3}}
 + -0.5 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{2}h_{3}}
 + 1.0 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{3}}
 + -0.5 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{2}h_{3}}
 + -0.5 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{2}h_{3}}
 + 1.0 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{2}h_{3}}
 + 0.5 * \Phi_{h_{3}h_{4}}^{p_{0}p_{1}}f_{h_{5}}^{h_{3}}t2_{p_{0}p_{1}}^{h_{4}h_{5}}
 + -1.0 * \Phi_{h_{3}h_{4}}^{p_{0}p_{1}}f_{h_{5}}^{h_{3}}t2_{p_{1}p_{0}}^{h_{4}h_{5}}
 + -1.0 * \Phi_{h_{3}h_{5}}^{p_{0}p_{1}}f_{h_{4}}^{h_{3}}t2_{p_{0}p_{1}}^{h_{4}h_{5}}
 + 0.5 * \Phi_{h_{3}h_{5}}^{p_{0}p_{1}}f_{h_{4}}^{h_{3}}t2_{p_

In [113]:
len(CCDDoublesResidual.summandList)

724

In [114]:
testCCDDoublesResidual = copy(CCDDoublesResidual)

In [115]:
print(testCCDDoublesResidual.summandList[0].vertexList[0])

\Phi_{h_{2}h_{3}}^{p_{0}p_{1}}


In [116]:
for summand in testCCDDoublesResidual.summandList:
    summand.vertexList.pop(0)

In [117]:
print(testCCDDoublesResidual)

-0.5 * f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{3}}
 + 1.0 * f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{2}h_{3}}
 + 1.0 * f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{2}h_{3}}
 + -0.5 * f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{2}h_{3}}
 + 1.0 * f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{3}}
 + -0.5 * f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{2}h_{3}}
 + -0.5 * f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{2}h_{3}}
 + 1.0 * f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{2}h_{3}}
 + 0.5 * f_{h_{5}}^{h_{3}}t2_{p_{0}p_{1}}^{h_{4}h_{5}}
 + -1.0 * f_{h_{5}}^{h_{3}}t2_{p_{1}p_{0}}^{h_{4}h_{5}}
 + -1.0 * f_{h_{4}}^{h_{3}}t2_{p_{0}p_{1}}^{h_{4}h_{5}}
 + 0.5 * f_{h_{4}}^{h_{3}}t2_{p_{1}p_{0}}^{h_{4}h_{5}}
 + -1.0 * f_{h_{5}}^{h_{3}}t2_{p_{0}p_{1}}^{h_{4}h_{5}}
 + 0.5 * f_{h_{5}}^{h_{3}}t2_{p_{1}p_{0}}^{h_{4}h_{5}}
 + 2.0 * f_{h_{3}}^{h_{3}}t2_{p_{0}p_{1}}^{h_{4}h_{5}}
 + -1.0 * f_{h_{3}}^{h_{3}}t2_{p_{1}p_{0}}^{h_{4}h_{5}}
 + 0.5 * f_{h_{4}}^{h_{3}}t2_{p_{0}p_{1}}^{h_{4}h_{5}}
 + -1.0 * f_{h_{4}}^{h_{3}}t2_{p_{1}p_{0}}^{h_{4}h_{5}}
 + -

In [190]:
CCDDoublesResidualSummandList = [s for s in (projectionManifold(2) * transformH).summandList if (s.vertexList[0].lowerIndices == [('h_{0}', True), ('h_{1}', True)] and s.vertexList[0].upperIndices == [('p_{0}', False), ('p_{1}', False)])]

In [191]:
CCDDoublesResidual = TensorSum(CCDDoublesResidualSummandList)
print(CCDDoublesResidual)

0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{2}}^{p_{3}}t2_{p_{4}p_{5}}^{h_{2}h_{3}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}f_{p_{4}}^{p_{5}}
 + 1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{2}}^{p_{3}}
 + 0.03125 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{2}}^{p_{3}}t2_{p_{4}p_{5}}^{h_{2}h_{3}}t2_{p_{6}p_{7}}^{h_{4}h_{5}}
 + -0.03125 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}f_{p_{4}}^{p_{5}}t2_{p_{6}p_{7}}^{h_{4}h_{5}}
 + -0.03125 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}f_{p_{4}}^{p_{5}}t2_{p_{6}p_{7}}^{h_{4}h_{5}}
 + 0.03125 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}t2_{p_{4}p_{5}}^{h_{4}h_{5}}f_{p_{6}}^{p_{7}}
 + 0.0026041666666666665 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{2}}^{p_{3}}t2_{p_{4}p_{5}}^{h_{2}h_{3}}t2_{p_{6}p_{7}}^{h_{4}h_{5}}t2_{p_{8}p_{9}}^{h_{6}h_{7}}
 + -0.0026041666666666665 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}f_{p_{4}}^{p_{5}}t2_{p_{6}p_{7}}^{h_{4}h_{5}}t2_{p_{8}p_{9

In [192]:
len(CCDDoublesResidualSummandList)

403

In [193]:
time0 = time.time()
contractedCCDDoublesResidual = evaluateWick(CCDDoublesResidual)
time1 = time.time()
print(time1-time0)
print(contractedCCDDoublesResidual)

2265.9064230309996
-0.5 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{3}}
 + 1.0 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{2}h_{3}}
 + 1.0 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{2}h_{3}}
 + -0.5 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{2}h_{3}}
 + 1.0 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{3}}
 + -0.5 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{2}h_{3}}
 + -0.5 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{2}h_{3}}
 + 1.0 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{2}h_{3}}
 + 0.5 * \Phi_{h_{3}h_{4}}^{p_{0}p_{1}}f_{h_{5}}^{h_{3}}t2_{p_{0}p_{1}}^{h_{4}h_{5}}
 + -1.0 * \Phi_{h_{3}h_{4}}^{p_{0}p_{1}}f_{h_{5}}^{h_{3}}t2_{p_{1}p_{0}}^{h_{4}h_{5}}
 + -1.0 * \Phi_{h_{3}h_{5}}^{p_{0}p_{1}}f_{h_{4}}^{h_{3}}t2_{p_{0}p_{1}}^{h_{4}h_{5}}
 + 0.5 * \Phi_{h_{3}h_{5}}^{p_{0}p_{1}}f_{h

In [141]:
testTensor1 = Tensor('test1', ['p','p'], ['h','h'])
testTensor2 = Tensor('test2', ['h','h'], ['p','p'])

In [142]:
testProduct = testTensor2 * testTensor1
print(testProduct)

1.0 * test2_{h_{0}h_{1}}^{p_{0}p_{1}}test1_{p_{2}p_{3}}^{h_{2}h_{3}}


In [143]:
testTensor2

In [147]:
testProduct.vertexList[0].tensor is testTensor2

True

In [152]:
testProduct.vertexList[0]

In [189]:
print(testProduct.vertexList[0].lowerIndices[0] == ('h_{0}', True))

False


In [148]:
testProductCopy = copy(testProduct)

In [154]:
testProductCopy.vertexList[0].tensor is testTensor2

True

In [155]:
testWick = evaluateWick(testProduct)
print(testWick)

4.0 * test2_{h_{2}h_{3}}^{p_{0}p_{1}}test1_{p_{0}p_{1}}^{h_{2}h_{3}}
 + -2.0 * test2_{h_{2}h_{3}}^{p_{0}p_{1}}test1_{p_{1}p_{0}}^{h_{2}h_{3}}
 + -2.0 * test2_{h_{3}h_{2}}^{p_{0}p_{1}}test1_{p_{0}p_{1}}^{h_{2}h_{3}}
 + 4.0 * test2_{h_{3}h_{2}}^{p_{0}p_{1}}test1_{p_{1}p_{0}}^{h_{2}h_{3}}


In [161]:
testTensor2.getShape(vacuum)

In [163]:
testTensor2

In [167]:
testWick.summandList[2].tensorList[0].array

array([[[[0.]]]])

In [168]:
testCommutator = commutator(testTensor1, testTensor2)
print(testCommutator)

1.0 * test1_{p_{0}p_{1}}^{h_{0}h_{1}}test2_{h_{2}h_{3}}^{p_{2}p_{3}}
 + -1 * test2_{h_{0}h_{1}}^{p_{0}p_{1}}test1_{p_{2}p_{3}}^{h_{2}h_{3}}


In [172]:
testCommutator.summandList[1].tensorList

[<__main__.Tensor at 0x1211d91c0>, <__main__.Tensor at 0x1211d92b0>]

In [120]:
def getAxis(vertex, index):
    for a in range(vertex.excitationRank):
        if vertex.lowerIndices[a] == index:
            return a
        elif vertex.upperIndices[a] == index:
            return vertex.excitationRank + a

In [121]:
print(testProduct.summandList[0].vertexList[0].lowerIndices[0])

h_{2}


In [122]:
getAxis(testProduct.summandList[0].vertexList[1], testProduct.summandList[0].vertexList[0].upperIndices[1])

1

In [133]:
testTensor1.getShape(vacuum)

In [140]:
testTensor2

In [139]:
testProduct.summandList[0].vertexList[0].tensor

In [144]:
np.einsum('ijkl,klij->', testProduct.summandList[0].vertexList[0].tensor.array, testProduct.summandList[0].vertexList[1].tensor.array)

0.0

In [145]:
np.einsum('ijkl,lkij->', testProduct.summandList[1].vertexList[0].tensor.array, testProduct.summandList[1].vertexList[1].tensor.array)

0.0

In [146]:
np.einsum('jikl,klij->', testProduct.summandList[2].vertexList[0].tensor.array, testProduct.summandList[2].vertexList[1].tensor.array)

0.0

In [147]:
np.einsum('jikl,lkij->', testProduct.summandList[2].vertexList[0].tensor.array, testProduct.summandList[2].vertexList[1].tensor.array)

0.0

In [207]:
CCDCorrelationEnergy = 0.
for summand in testProduct.summandList:
    einsumString = 'abcd,'
    for i, index in enumerate(summand.vertexList[1].lowerIndices + summand.vertexList[1].upperIndices):
       einsumString += einsumString[getAxis(summand.vertexList[0], index)]
    einsumString += '->'
    print(einsumString)
    contribution = summand.prefactor * np.einsum(einsumString, summand.vertexList[0].tensor.array, summand.vertexList[1].tensor.array)
    print(contribution)
    CCDCorrelationEnergy += contribution
print(CCDCorrelationEnergy)

abcd,cdab->
-0.022207332862524486
abcd,dcab->
0.011103666431262243
abcd,cdba->
0.011103666431262243
abcd,dcba->
-0.022207332862524486
-0.022207332862524486


In [247]:
eri

array([[6.54009511e-01, 1.11022302e-16, 8.32667268e-17, 6.45249427e-01],
       [1.17961196e-16, 1.87521981e-01, 1.87521981e-01, 1.11022302e-16],
       [1.04083409e-16, 1.87521981e-01, 1.87521981e-01, 5.55111512e-17],
       [6.45249427e-01, 2.22044605e-16, 1.38777878e-16, 6.78136184e-01]])

In [248]:
integrals2body = eri.reshape((Norbs,Norbs,Norbs,Norbs))

In [191]:
integrals2body

array([[[[6.54009511e-01, 1.11022302e-16],
         [8.32667268e-17, 6.45249427e-01]],

        [[1.17961196e-16, 1.87521981e-01],
         [1.87521981e-01, 1.11022302e-16]]],


       [[[1.04083409e-16, 1.87521981e-01],
         [1.87521981e-01, 5.55111512e-17]],

        [[6.45249427e-01, 2.22044605e-16],
         [1.38777878e-16, 6.78136184e-01]]]])

In [192]:
print(vDiagrams[8])

v_{hh}^{pp}


In [249]:
vDiagrams[8].array = integrals2body[:Nocc, :Nocc, Nocc:, Nocc:]

In [171]:
integrals2body[:Nocc, :Nocc, Nocc:, Nocc:]

array([[[[0.64524943]]]])

In [194]:
vDiagrams[8].array

array([[[[0.64524943]]]])

In [251]:
t2.getShape(vacuum)

In [252]:
t2.array[0,0,0,0] = -0.06883332846938728

In [244]:
for diagram in vDiagrams:
    diagram.getShape(vacuum)

In [198]:
testProduct = evaluateWick(0.5 * vDiagrams[8] * 0.25 * t2)

In [200]:
print(testProduct)

0.5 * v_{h_{2}h_{3}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{3}}
 + -0.25 * v_{h_{2}h_{3}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{2}h_{3}}
 + -0.25 * v_{h_{3}h_{2}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{3}}
 + 0.5 * v_{h_{3}h_{2}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{2}h_{3}}


In [202]:
CCDCorrelationEnergy + mf.e_tot

-1.1248461440542483

In [187]:
cisolver.kernel()

(-1.1284543355083052,
 array([[ 9.90656547e-01,  2.77555756e-17],
        [ 1.91460513e-17, -1.36380375e-01]]))

In [209]:
testProduct.summandList[0].lowerIndices

{'g': [],
 'p': [<__main__.Index at 0x126e896a0>, <__main__.Index at 0x126e89790>],
 'h': [<__main__.Index at 0x126e898b0>, <__main__.Index at 0x126e89940>],
 'a': []}

In [224]:
import string
import itertools

In [225]:
indexLetters = string.ascii_lowercase

In [234]:
CCDCorrelationEnergy = 0.
for summand in testProduct.summandList:
    lowerIndexList = list(itertools.chain.from_iterable([vertex.lowerIndices for vertex in summand.vertexList]))
    upperIndexList = list(itertools.chain.from_iterable([vertex.upperIndices for vertex in summand.vertexList]))
    for index in upperIndexList:
        print(index)
    for index in lowerIndexList:
        print(index)
    lowerIndexLetters = indexLetters[:sum([vertex.excitationRank for vertex in summand.vertexList])]
    upperIndexLetters = ''
    for lowerIndex in lowerIndexList:
        for uI, upperIndex in enumerate(upperIndexList):
            if upperIndex == lowerIndex:
                upperIndexLetters += lowerIndexLetters[uI]
    print(upperIndexLetters)
    print(lowerIndexLetters)
    einsumSubstrings = []
    start = 0
    for vertex in summand.vertexList:
        end = start + vertex.excitationRank
        einsumSubstring = lowerIndexLetters[start:end] + upperIndexLetters[start:end]
        einsumSubstrings.append(einsumSubstring)
        start = end
    einsumString = ''
    for sub in range(len(einsumSubstrings) - 1):
        einsumString += einsumSubstrings[sub] + ','
    einsumString += einsumSubstrings[-1] + '->'
    print(einsumString)
    contribution = summand.prefactor * np.einsum(einsumString, *[vertex.tensor.array for vertex in summand.vertexList])
    print(contribution)
    CCDCorrelationEnergy += contribution
print(CCDCorrelationEnergy)

p_{0}
p_{1}
h_{2}
h_{3}
h_{2}
h_{3}
p_{0}
p_{1}
cdab
abcd
abcd,cdab->
-0.022207332862524486
p_{0}
p_{1}
h_{2}
h_{3}
h_{2}
h_{3}
p_{1}
p_{0}
cdba
abcd
abcd,cdba->
0.011103666431262243
p_{0}
p_{1}
h_{2}
h_{3}
h_{3}
h_{2}
p_{0}
p_{1}
dcab
abcd
abdc,cdab->
0.011103666431262243
p_{0}
p_{1}
h_{2}
h_{3}
h_{3}
h_{2}
p_{1}
p_{0}
dcba
abcd
abdc,cdba->
-0.022207332862524486
-0.022207332862524486


In [232]:
testRes = testCCDDoublesResidual.summandList[0]

In [240]:
print(testRes)

-0.5 * f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{3}}


In [226]:
def getContractedArray(operatorProduct_):
    lowerIndexList = list(itertools.chain.from_iterable([vertex.lowerIndices for vertex in operatorProduct_.vertexList]))
    upperIndexList = list(itertools.chain.from_iterable([vertex.upperIndices for vertex in operatorProduct_.vertexList]))
#    for index in lowerIndexList:
#        print(index)
#    for index in upperIndexList:
#        print(index)
    lowerIndexLetters = string.ascii_lowercase[:len(lowerIndexList)]
    upperIndexLetters = ''
    freeLowerIndexMask = np.ones(len(lowerIndexList))
    freeUpperIndexMask = np.ones(len(upperIndexList))
    nFreeUpperIndices = 0
    for uI, upperIndex in enumerate(upperIndexList):
        free = True
        for lI, lowerIndex in enumerate(lowerIndexList):
            if upperIndex == lowerIndex:
                upperIndexLetters += lowerIndexLetters[lI]
                freeLowerIndexMask[lI] = 0
                freeUpperIndexMask[uI] = 0
                free = False
        if free:
            upperIndexLetters += string.ascii_lowercase[len(lowerIndexList) + nFreeUpperIndices]
            nFreeUpperIndices += 1
#    print(lowerIndexLetters)
#    print(upperIndexLetters)
    freeLowerIndexLetters = "".join([lowerIndex for lI, lowerIndex in enumerate(lowerIndexLetters) if freeLowerIndexMask[lI]])
    freeUpperIndexLetters = "".join([upperIndex for uI, upperIndex in enumerate(upperIndexLetters) if freeUpperIndexMask[uI]])
#    print(freeLowerIndexLetters)
#    print(freeUpperIndexLetters)
    einsumSubstrings = []
    start = 0
    for vertex in operatorProduct_.vertexList:
        end = start + vertex.excitationRank
        einsumSubstring = lowerIndexLetters[start:end] + upperIndexLetters[start:end]
        einsumSubstrings.append(einsumSubstring)
        start = end
    einsumString = ",".join(einsumSubstrings)
    einsumString += '->' + freeLowerIndexLetters + freeUpperIndexLetters
#    print(einsumString)
    print([vertex.tensor.array.shape for vertex in operatorProduct_.vertexList])
    contribution = operatorProduct_.prefactor * np.einsum(einsumString, *[vertex.tensor.array for vertex in operatorProduct_.vertexList])
#    print(contribution)
    return contribution

In [308]:
for s, summand in enumerate(testCCDDoublesResidual.summandList):
    if (getContractedArray(summand).shape != getContractedArray(testCCDDoublesResidual.summandList[0]).shape):
        print(s)

In [321]:
print(CCDDoublesResidual.summandList[1])

1.0 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{2}h_{3}}


In [255]:
for t in CCDEnergy.summandList:
    print(t)
    print(getContractedArray(t))

2.0 * f_{h_{1}}^{h_{1}}
[(1, 1)]
0.0
-1.0 * v_{h_{2}h_{3}}^{h_{2}h_{3}}
[(1, 1, 1, 1)]
-0.0
2.0 * v_{h_{3}h_{2}}^{h_{2}h_{3}}
[(1, 1, 1, 1)]
0.0
0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{1}}
[(1, 1, 1, 1), (1, 1, 1, 1)]
-0.022207332862524486
-0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{1}}
[(1, 1, 1, 1), (1, 1, 1, 1)]
0.011103666431262243
-0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{1}h_{0}}
[(1, 1, 1, 1), (1, 1, 1, 1)]
0.011103666431262243
0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{1}h_{0}}
[(1, 1, 1, 1), (1, 1, 1, 1)]
-0.022207332862524486


In [231]:
print(getContractedArray(testCCDDoublesResidual.summandList[0]))

AttributeError: 'Tensor' object has no attribute 'array'

In [329]:
print(testCCDDoublesResidual.summandList[0].vertexList[1].tensor.array)

[[[[0. 0.]
   [0. 0.]]]]


In [219]:
t2.getShape(vacuum)
t2.setArray(np.array([[[[-0.06883333]]]]))

In [307]:
testCCDDoublesResidual.summandList.pop(3)

In [221]:
doublesResidualArray = sum([getContractedArray(s) for s in testCCDDoublesResidual.summandList])

NameError: name 'getContractedArray' is not defined

In [310]:
doublesResidualArray

array([[[[0., 0.],
         [0., 0.]]]])

In [311]:
doublesResidualTensor = Tensor('R', ['p', 'p'], ['h', 'h'], array=doublesResidualArray)

Array is of wrong shape


In [220]:
t2.array

array([[[[-0.06883333]]]])

In [319]:
print(evaluateWick(projectionDoubles * t2))

4.0 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{3}}
 + -2.0 * \Phi_{h_{2}h_{3}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{2}h_{3}}
 + -2.0 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{3}}
 + 4.0 * \Phi_{h_{3}h_{2}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{2}h_{3}}


In [331]:
testCCDDoublesResidual.summandList[0].vertexList[0].tensor

In [332]:
print(testCCDDoublesResidual.summandList[0])

-0.5 * f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{3}}


In [387]:
print(CCDEnergy)

2.0 * f_{h_{1}}^{h_{1}}
 + -1.0 * v_{h_{2}h_{3}}^{h_{2}h_{3}}
 + 2.0 * v_{h_{3}h_{2}}^{h_{2}h_{3}}
 + 0.5 * v_{h_{2}h_{3}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{3}}
 + -0.25 * v_{h_{2}h_{3}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{2}h_{3}}
 + -0.25 * v_{h_{3}h_{2}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{3}}
 + 0.5 * v_{h_{3}h_{2}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{2}h_{3}}


In [395]:
CCDEnergy.summandList[3].vertexList[1].tensor = t2

In [358]:
print(fDiagrams[3])

f_{h}^{h}


In [357]:
h1

array([[-1.18985062e+00,  2.60021255e-17],
       [-4.71423569e-17, -5.33749102e-01]])

In [359]:
h1[:1,:1]

array([[-1.18985062]])

In [377]:
print(vDiagrams[8].array)

[[[[0.64524943]]]]


In [375]:
vDiagrams[1].array = integrals2body[:1,:1,:1,:1]

In [364]:
fDiagrams[3] = Tensor('f', ['h'], ['h'], h1[:1,:1])

In [366]:
print(CCDEnergy.summandList[0].vertexList[0].tensor.array)

[[-1.18985062]]


In [234]:
CCDEnergy.summandList[0].tensorList

In [235]:
fDiagrams

In [232]:
print(sum([getContractedArray(s) for s in CCDEnergy.summandList]))

AttributeError: 'Tensor' object has no attribute 'array'

In [401]:
mol.energy_nuc() + sum([getContractedArray(s) for s in CCDEnergy.summandList[:3]])

[(1, 1)]
[(1, 1, 1, 1)]
[(1, 1, 1, 1)]


-1.1026388111917238

In [402]:
mf.e_tot

-1.1026388111917238